In [ ]:
#imports
import optuna
from model import *

## Optuna Function
def objective(trial):
    # Suggest hyperparameters
    # Restrict d_model and nhead to best from first optuna to get better insights (no inf)
    d_model = 64
    nhead = 4

    params = {
        'learning_rate': trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_int("batch_size", 8, 64),
        'seq_length': trial.suggest_int("seq_length", 3, 21),
        'transformer_encoder_layer_params': {
            'd_model': d_model,
            'nhead': nhead,
            'dim_feedforward': trial.suggest_int("dim_feedforward", 128, 512),
            'dropout': trial.suggest_float("dropout", 0.0, 0.1),
            'activation': 'relu'
        },
        'transformer_layer_params': {
            'num_layers': trial.suggest_int("num_layers", 1, 5),
        },
        'dataset': '2010-2019',
        'train_test_split': '80:20',
        'features': ['sum_30_min_demand', 'is_weekend', 'is_weekday'],
    }

    return median_mape(params)

In [2]:
study = optuna.create_study(storage="sqlite:///db.sqlite3", study_name="weekday_weekend", load_if_exists=True)
study.optimize(objective, n_trials=100, n_jobs=1)

study.best_params 

[I 2025-09-30 17:25:27,184] A new study created in RDB with name: weekday_weekend
[W 2025-09-30 17:25:31,232] Trial 0 failed with parameters: {'learning_rate': 0.00023398216584146375, 'batch_size': 15, 'seq_length': 9, 'dim_feedforward': 275, 'dropout': 0.016473407435439513, 'num_layers': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/harry/personal/uni/project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_967798/1881916518.py", line 39, in objective
    model, train_losses, val_losses = train_transformer_model(sequences, targets, input_dim, datetimes, params)
  File "/home/harry/personal/uni/project/individual/harry/model/model.py", line 176, in train_transformer_model
    predictions = model(batch_X)
  File "/home/harry/personal/uni/project/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call

KeyboardInterrupt: 

In [84]:
study.best_params

{'dropout': 0.1503743360111539,
 'learning_rate': 0.0011392910510892803,
 'batch_size': 32,
 'seq_length': 21,
 'dim_feedforward': 339,
 'activation': 'relu',
 'num_layers': 2}